# Road Scene Understanding with Kitti Dataset
#### CS 5190
#### Team Members: 

## SETUP
> conda install matplotlib numpy opencv pandas scikit-image scikit-learn scipy ultralytics opencv-python tqdm pillow

Each line in a label file contains the following:
`<object_type> <truncation> <occlusion> <alpha> <left> <top> <right> <bottom> <height> <width> <length> <x> <y> <z> <rotation_y>`

YOLOV8 requires format of 'class x_center y_center width height' -> for each image has one txt file with a single line for each bounding box.
Structure for yolov8: https://roboflow.com/formats/yolov8-pytorch-txt

# Dataset

In [ ]:
# KITTI BASE PATHS
full_path_project_folder = INSERT_PATH_HERE
base_yolo_path = full_path_project_folder + "finalProject\\yolov8\\"
base_labels_path = full_path_project_folder + "finalProject\\data_object_label_2\\training\\label_2\\"
base_images_train_path = full_path_project_folder + "finalProject\\data_object_image_2\\training\\image_2\\"

# KITTI CLASSES/OBJECT types in the labels file
OBJECT_CLASSES = {'Car': 0, 'Van': 1, 'Truck': 2, 'Pedestrian': 3, 'Person_sitting': 4, 'Cyclist': 5, 'Tram': 6, 'Misc': 7, 'DontCare': 8}

In [ ]:
import os
from PIL import Image

#create pairs for the paths to KITTI images and labels
path_pairs = []
if not os.path.isdir(base_labels_path):
    print(f"Error: Folder {base_labels_path} not found")
elif not os.path.isdir(base_images_train_path):
    print(f"Error: Folder {base_images_train_path} not found")
else:
    for full_filename in os.listdir(base_labels_path):
        filename = full_filename.split('.')
        pair = {"img_path": (base_images_train_path + filename[0] + ".png"), "label_path": (base_labels_path + filename[0] + ".txt") }
        path_pairs.append(pair)

In [ ]:
#TODO: Might want to implement train split later on
from sklearn.model_selection import train_test_split

#seperate into 80% training and 20% validation
train, validate = train_test_split(path_pairs, test_size=0.2, shuffle=True)


In [ ]:
#print a few to see if it worked right
path_pairs[:2]

[{'img_path': 'C:\\Users\\Sabrina Ferras\\OneDrive - csumb.edu\\Documents\\CPP\\4Fall2025\\CS_5190\\finalProject\\data_object_image_2\\training\\image_2\\000000.png',
  'label_path': 'C:\\Users\\Sabrina Ferras\\OneDrive - csumb.edu\\Documents\\CPP\\4Fall2025\\CS_5190\\finalProject\\data_object_label_2\\training\\label_2\\000000.txt'},
 {'img_path': 'C:\\Users\\Sabrina Ferras\\OneDrive - csumb.edu\\Documents\\CPP\\4Fall2025\\CS_5190\\finalProject\\data_object_image_2\\training\\image_2\\000001.png',
  'label_path': 'C:\\Users\\Sabrina Ferras\\OneDrive - csumb.edu\\Documents\\CPP\\4Fall2025\\CS_5190\\finalProject\\data_object_label_2\\training\\label_2\\000001.txt'}]

## Preprocessing

In [ ]:
#function that takes in bounding box coordinates from the label file and image width&height 
#return x_center, y_center, width, and height
def convert_bbox_yolo8(img_w, img_h, x1 , y1, x2, y2):
    x_center = ((x1 + x2) / 2 ) / img_w
    y_center = ((y1 + y2) / 2 ) / img_h
    width = (x2 - x1) / img_w
    height = (y2 - y1) / img_h

    return x_center, y_center, width, height

In [ ]:
for curr_pair in path_pairs:
    #set yolo file
    yolo_path = base_yolo_path + os.path.basename(curr_pair['label_path'])
    
    #open image to get width & height
    try:
        img = Image.open(curr_pair['img_path'])
        img_width, img_height = img.size
    except Exception as e:
        print(f"Error opening image {curr_pair['img_path']}: {e}")
        continue
    
    #loop through label file line by line to add to yolo .txt file
    with open(curr_pair['label_path']) as f:
        lines = f.readlines()
    
    yolo_lines = []
    for line in lines:
        label_parts = line.strip().split()
        object_name = label_parts[0]
        class_id = OBJECT_CLASSES[object_name]
        if class_id == 8: continue # excluding the DontCare bounding boxes
        bb_x1, bb_y1, bb_x2, bb_y2 = map(float, label_parts[4:8])   #taking original bounding box coordinates from kitti label
        x_center, y_center, width, height = convert_bbox_yolo8(img_width, img_height, bb_x1 , bb_y1, bb_x2, bb_y2) 
        yolo_lines.append(f"{class_id} {x_center} {y_center} {width} {height}\n") #yolo format

    #create and write yolo normalized bounding box
    with open(yolo_path, "w") as out:
        out.writelines(yolo_lines)

## Model Training

## Testing

## Visualizations